# Roman Numeral Calculator


In [ ]:
# Import the json library to read Roman numeral mapping files
import json

## Calculator Class

In [ ]:
class Calculator():
    """
    A basic calculator class that performs arithmetic operations on two numbers.
    """
    def __init__(self, a, b):
        """Initialize the calculator with two numbers."""
        self.a = a
        self.b = b

    def add(self):
        """Return the sum of the two numbers."""
        return self.a + self.b
    
    def sub(self):
        """Return the difference of the two numbers."""
        return self.a - self.b
    
    def mul(self):
        """Return the product of the two numbers."""
        return self.a * self.b
    
    def __str__(self):
        """String representation of the calculator."""
        return f"A: {self.a}\nB: {self.b}"

## RomanToNumber Class

In [ ]:
class RomanToNumber():
    """
    Converts Roman numerals to decimal numbers.
    Handles single characters, subtraction pairs, and complex Roman numerals.
    """
    def __init__(self, a):
        """
        Initialize the converter with a Roman numeral string.
        Loads mapping data from JSON files.
        """
        # Convert to uppercase and split into list of characters
        self.a = list(a.upper())

        # Load basic Roman numeral to number mappings (I=1, V=5, X=10, etc.)
        with open("roman.json", 'r') as f:
            self.romans = json.load(f)
        
        # Load subtraction pair rules (IV=4, IX=9, XL=40, etc.)
        with open("substraction.json", "r") as f:
            self.substraction_pairs = json.load(f)
        
    def to_num(self):
        """
        Convert the Roman numeral to a decimal number.
        Returns: Integer value of the Roman numeral
        """
        number = 0

        # Case 1: Single character (e.g., "I", "V", "X")
        if len(self.a) == 1:
            number = self.romans[self.a[0]]

        # Case 2: Two characters (check for subtraction pair)
        elif len(self.a) == 2:
            pair = self.a[0] + self.a[1]
            # Check if it's a subtraction pair like "IV" or "IX"
            if pair in self.substraction_pairs["Substraction_rule"]:
                index = self.substraction_pairs["Substraction_rule"].index(pair)
                number = self.substraction_pairs["Substituting_values"][index]
            else:
                # Just add both values (e.g., "VI" = 5 + 1)
                for i in self.a:
                    number += self.romans[i]

        # Case 3: Three or more characters
        else:
            # Add all characters except the last two
            for i in self.a[:-2]:
                number += self.romans[i]
            
            # Check if last two characters form a subtraction pair
            pair = self.a[-2] + self.a[-1]
            if pair in self.substraction_pairs["Substraction_rule"]:
                index = self.substraction_pairs["Substraction_rule"].index(pair)
                number += self.substraction_pairs["Substituting_values"][index]
            else:
                # Add last two characters separately
                number += self.romans[self.a[-2]]
                number += self.romans[self.a[-1]]
        
        return number

    def __str__(self):
        """String representation of the Roman numeral."""
        return f"Roman number is: {self.a}"

## NumberToRoman Class

In [ ]:
class NumberToRoman():
    """
    Converts decimal numbers to Roman numerals.
    Handles numbers from 1 to beyond 3999 using vinculum notation.
    """
    def __init__(self, a):
        """
        Initialize the converter with a decimal number.
        Loads mapping data from JSON files.
        """
        self.a = str(a)
    
        # Load subtraction pair data
        with open("substraction.json", "r") as f:
            self.substraction_pairs = json.load(f)
        
        # Load basic Roman numeral mappings
        with open("roman.json", 'r') as f:
            self.romans = json.load(f)
        
        # Load single digit to Roman numeral mappings (1-9)
        with open("unit_numbers.json") as f:
            self.unit_romans = json.load(f)
        
    def add_vinculum(self, string):
        """
        Add vinculum (overline) to Roman numerals for numbers > 3999.
        Vinculum multiplies the value by 1000.
        Example: V̅ = 5000
        """
        return "".join([char + '\u0305' for char in string])
    
    def to_roman_helper(self, number):
        """
        Helper function to convert remainders to Roman numerals.
        Handles hundreds (100-200) and tens (10-30).
        """
        roman_number = str()
        roman_list = []

        # Handle hundreds: 100 (C) or 200 (CC)
        if 1 <= (number // 100) < 3:
            count = number // 100
            for _ in range(count):
                roman_list.append("C")
        
        # Handle tens: 10-30 (X, XX, XXX)
        elif 1 <= (number // 10) < 4:
            count = number // 10
            for _ in range(count):
                roman_list.append("X")
        
        roman_number = ''.join(roman_list)
        return roman_number
    
    def to_roman(self):
        """
        Convert the decimal number to Roman numerals.
        Returns: String representation of the Roman numeral
        """
        length = len(self.a)
        num_list = list(self.a)
        places_list = []  # Store each digit's place value (e.g., 3888 -> [3000, 800, 80, 8])
        roman_list = []   # Store Roman numeral components

        # Handle single digit numbers (1-9)
        if length == 1:
            return self.unit_romans[self.a]
        
        # Break number into place values
        # Example: 3888 -> [3000, 800, 80, 8]
        for i in num_list:
            places_list.append(int(i) * (10**(length-1)))
            length -= 1
        
        # Convert each place value to Roman numerals
        for i in places_list:
            # Handle thousands (1000-3000 or higher with vinculum)
            if len(str(i)) >= 3 and i // 1000:
                if 1 < i // 1000 < 4:
                    # 2000 (MM) or 3000 (MMM)
                    for _ in range(i // 1000):
                        roman_list.append("M")
                else:
                    # Numbers >= 4000 use vinculum notation
                    a = i // 1000
                    roman_list.append(self.add_vinculum(self.unit_romans[str(a)]))
            
            # Handle 500-900 range
            elif len(str(i)) == 3 and i // 500:
                if i // 500:
                    remainder = i - 500
                    # 500-800: D, DC, DCC, DCCC
                    if remainder // 100 <= 3:
                        roman_numeral_remainder = self.to_roman_helper(remainder)
                        roman_list.append("D" + roman_numeral_remainder)
                    else:
                        # 900: CM
                        roman_list.append("CM")

            # Handle 100-400 range
            elif len(str(i)) == 3 and i // 100:
                if i // 100:
                    remainder = i - 100
                    # 100-300: C, CC, CCC
                    if remainder // 100 <= 2:
                        roman_numeral_remainder = self.to_roman_helper(remainder)
                        roman_list.append("C" + roman_numeral_remainder)
                    else:
                        # 400: CD
                        roman_list.append("CD")

            # Handle tens (10-90)
            elif len(str(i)) == 2: 
                if i // 50:
                    # 50-80: L, LX, LXX, LXXX
                    remainder = i - 50
                    if remainder // 10 <= 3:
                        roman_numeral_remainder = self.to_roman_helper(remainder)
                        roman_list.append("L" + roman_numeral_remainder)
                    else:
                        # 90: XC
                        roman_list.append("XC")
                else:
                    # 10-40: X, XX, XXX, or XL
                    remainder = i - 10
                    if remainder // 10 <= 2:
                        roman_numeral_remainder = self.to_roman_helper(remainder)
                        roman_list.append("X" + roman_numeral_remainder)
                    else:
                        # 40: XL
                        roman_list.append("XL")
            
            # Handle units (1-9)
            elif len(str(i)) == 1 and i != 0:
                roman_list.append(self.unit_romans[str(i)])
        
        # Join all components into final Roman numeral
        roman_number = ''.join(roman_list)
        return roman_number

## Interactive Roman Calculator

In [ ]:
# Interactive Roman Calculator
# Get user input for the first Roman numeral and convert to decimal
a = RomanToNumber(input("Enter 1st Roman Number: ")).to_num()

# Get user input for the second Roman numeral and convert to decimal
b = RomanToNumber(input("Enter 2nd Roman Number: ")).to_num()

# Get the operator from user
operator = input("\nEnter the Operator (+, -, *): ")

# Create a calculator instance with the two decimal numbers
solution = Calculator(a, b)

# Perform the calculation based on the operator
if operator == "+":
    result = solution.add()
elif operator == '-':
    result = solution.sub()
elif operator == '*':
    result = solution.mul()
else:
    print("Use only *, -, +")
    result = None

# Convert the result back to Roman numeral and display
if result is not None:
    converter = NumberToRoman(result)
    print(f"\nResult: {converter.to_roman()}")


Result: XXII


## Example Test Cases

### Example 1: Addition (XIV + VI)

In [ ]:
# Example 1: Addition - XIV (14) + VI (6) = XX (20)
num1 = RomanToNumber("XIV").to_num()  # Convert XIV to 14
num2 = RomanToNumber("VI").to_num()    # Convert VI to 6
calc = Calculator(num1, num2)          # Create calculator instance
result = calc.add()                     # Perform addition: 14 + 6 = 20
roman_result = NumberToRoman(result).to_roman()  # Convert 20 to XX
print(f"XIV + VI = {roman_result} ({num1} + {num2} = {result})")

XIV + VI = XX (14 + 6 = 20)


### Example 2: Subtraction (L - XX)

In [ ]:
# Example 2: Subtraction - L (50) - XX (20) = XXX (30)
num1 = RomanToNumber("L").to_num()     # Convert L to 50
num2 = RomanToNumber("XX").to_num()    # Convert XX to 20
calc = Calculator(num1, num2)          # Create calculator instance
result = calc.sub()                     # Perform subtraction: 50 - 20 = 30
roman_result = NumberToRoman(result).to_roman()  # Convert 30 to XXX
print(f"L - XX = {roman_result} ({num1} - {num2} = {result})")

L - XX = XXX (50 - 20 = 30)


### Example 3: Multiplication (XII * III)

In [ ]:
# Example 3: Multiplication - XII (12) * III (3) = XXXVI (36)
num1 = RomanToNumber("XII").to_num()   # Convert XII to 12
num2 = RomanToNumber("III").to_num()   # Convert III to 3
calc = Calculator(num1, num2)          # Create calculator instance
result = calc.mul()                     # Perform multiplication: 12 * 3 = 36
roman_result = NumberToRoman(result).to_roman()  # Convert 36 to XXXVI
print(f"XII * III = {roman_result} ({num1} * {num2} = {result})")

XII * III = XXXVI (12 * 3 = 36)


### Example 4: Large Number Conversion

In [ ]:
# Example 4: Large Number Conversion and Verification
# Convert 3888 to Roman numerals
# Expected: MMMDCCCLXXXVIII (MMM=3000, DCCC=800, LXXX=80, VIII=8)
num = 3888
roman = NumberToRoman(num).to_roman()
print(f"{num} in Roman numerals: {roman}")

# Verify by converting back to decimal
decimal = RomanToNumber(roman).to_num()
print(f"{roman} back to decimal: {decimal}")
print(f"Conversion successful: {decimal == num}")

3888 in Roman numerals: MMMDLXXXVIII
MMMDLXXXVIII back to decimal: 3588
